# ISM Effect

In [1]:
import numpy as np
from decimal import Decimal, getcontext
import Lya_zelda as Lya
import itertools
import h5py

from astropy import units as u
from astropy.cosmology import FlatLambdaCDM


In [2]:
# Define cosmology and units using astropy
cosmo = FlatLambdaCDM(H0=67.77, Om0=0.307115)
littleh = cosmo.H0.value / 100.0
Mpcph = u.def_unit('Mpcph', u.Mpc / littleh)  # [Mpc/h]
Msunph = u.def_unit('Msunph', u.Msun / littleh)  # [Msun/h]
kpcph = u.def_unit('kpcph', u.kpc / littleh)  # [kpc/h]

In [3]:
# load the data including scaling relation and luminosiy cut.

dat = np.load("sfr_catalog_0.258000_w_scaling_relations_masked_L41_pos400.npy", allow_pickle=True)


In [4]:
def calculate_v_exp(alpha, data, h5_filename):
    """
    Calculates the v_exp using the parametrization: v_exp = alpha * SFR * (r/sm)
    and saves into a .h5 file
    
    Inputs:
    -------
    alpha --> free parameter
    data --> load the catalog to get sfr, sm and galaxy size.
    
    Returns:
    --------
    v_exp --> expansion velocity of the thin shell. (array: (shape=(N,)) --> [unit: km/s]
    
    """
    sm = data["sm"]
    sfr = data["sfr"]
    size = data["size"]

    myr = 10 ** size
    sm_h = ((sm * u.Msun).to(Msunph)).value
    sfr_h_s = ((sfr * u.Msun / u.yr).to(Msunph / u.s)).value
    myr_km = ((myr * u.kpc).to(u.km)).value
    v_exp = alpha * sfr_h_s * (myr_km / sm_h)

    # Save the result in the HDF5 file with alpha and beta information
    with h5py.File(h5_filename, 'w') as file:
        # Create a group with the current alpha value
        group_name = f'alpha_{alpha}'
        group = file.create_group(group_name)

        # Save the v_exp dataset within the group
        dset_v_exp = group.create_dataset('v_exp_Arr', data=v_exp, compression="gzip", compression_opts=9, chunks=True, maxshape=(None,))

        # Add attributes to the group to store alpha value
        group.attrs['alpha'] = alpha

    #print('vexp is done!')
    return v_exp


In [5]:
def calculate_N_HI(alpha, beta, data, h5_filename):
    """
    Calculates the N_HI using the parametrization: N_HI = beta * (mcold/4*pi*m_H*r**2)
    and saves into a .h5 file
    
    Inputs:
    -------
    alpha --> free parameter
    beta --> free parameter
    data --> load the catalog to get mcold and zcold.
    
    Returns:
    --------
    N_HI --> column density of hydrogen of the thin shell. (array: (shape=(N,)) --> [unit: 1/cm2]
    
    """
    mcold = data["mcold"]
    zcold = data["zcold"]
    sample_M_cold_not_log = 10 ** mcold
    # Vectorize the Decimal conversion function
    decimal_converter = np.vectorize(lambda x: Decimal(str(x)))
    sample_M_cold_not_log_decimal = decimal_converter(sample_M_cold_not_log)
    result_decimal = sample_M_cold_not_log_decimal * Decimal("1.9891e30")
    mcold_nolog = np.array(result_decimal, dtype=float)
    M_c = (mcold_nolog / 0.70)
    myr = 10 ** data["size"]
    myr_km = ((myr * u.kpc).to(u.km)).value
    myr_cm = ((myr_km * u.km).to(u.cm)).value
    # Use NumPy vectorized operations for the remaining calculations
    getcontext().prec = 50
    rrr_decimal = np.array(myr_cm, dtype=object)  # Using dtype=object to handle Decimal
    rrr_decimal = np.vectorize(Decimal)(rrr_decimal)
    result_decimal_rrr = 1 / rrr_decimal ** 2
    myr_cm_new = np.array(result_decimal_rrr, dtype=float)
    N_HI_new = beta * (M_c * myr_cm_new)

    # Save the result in the HDF5 file with alpha and beta information
    with h5py.File(h5_filename, 'w') as file:
        # Create a group with the current alpha and beta values
        group_name = f'alpha_{alpha}_beta_{beta}'
        group = file.create_group(group_name)

        # Save the N_HI dataset within the group
        dset_N_HI = group.create_dataset('N_HI_Arr', data=N_HI_new, compression="gzip", compression_opts=9, chunks=True, maxshape=(None,))

        # Add attributes to the group to store alpha and beta values
        group.attrs['alpha'] = alpha
        group.attrs['beta'] = beta

    #print('NHI is done!')
    return N_HI_new



In [6]:
def calculate_tau(alpha, beta, data, h5_filename):
    
    """
    Calculates the tau (dust optical depth) using the parametrization:  (1 - A_Lya) * (E_Sun / Z_Sun) * N_HI * Z_c
    and saves into a .h5 file
    
    Inputs:
    -------
    alpha --> free parameter
    beta --> free parameter
    data --> load the catalog to get mcold and zcold.
    
    Returns:
    --------
    tau_a --> dust optical depth. (array: (shape=(N,))
    
    """
    
    N_HI_new = calculate_N_HI(alpha, beta, data, h5_filename)  # Pass the h5_filename to calculate_N_HI

    A_Lya = 0.39
    E_Sun = 1.77e-21
    Z_Sun = 0.02
    Z_c = 10 ** data["zcold"]
    tau_new = (1 - A_Lya) * (E_Sun / Z_Sun) * N_HI_new * Z_c

    # Save the result in the HDF5 file with alpha and beta information
    with h5py.File(h5_filename, 'a') as file:  # Open the file in 'a' (append) mode
        # Get the existing group with the current alpha and beta values
        group_name = f'alpha_{alpha}_beta_{beta}'
        group = file[group_name]

        # Save the tau dataset within the group
        dset_tau = group.create_dataset('tau_Arr', data=tau_new, compression="gzip", compression_opts=9, chunks=True, maxshape=(None,))

    #print('tau is done!')
    return tau_new


In [7]:
def calculate_f_esc(alpha, beta, data, h5_filename):
    """
    Calculates the tau (Dust optical depth) using the parametrization:  (1 - A_Lya) * (E_Sun / Z_Sun) * N_HI * Z_c
    and saves into a .h5 file
    
    Here we use zELDA code (written by Siddhartha Gurung-Lopez). In your environment do the following before using this function: 
    pip install zelda 
    and download the Grids as follows:
    curl --cookie zenodo-cookies.txt "https://zenodo.org/record/4733518/files/Grids.zip?download=1" --output Grids.zip
    
    Inputs:
    -------
    alpha --> free parameter
    beta --> free parameter
    data --> load the catalog to get mcold and zcold.
    
    Returns:
    --------
    tau_a --> dust optical depth. (array: (shape=(N,))
    
    """
    vexp = calculate_v_exp(alpha, data, h5_filename)
    N_HI = calculate_N_HI(alpha, beta, data, h5_filename)  # Pass the h5_filename to calculate_N_HI
    tau = calculate_tau(alpha, beta, data, h5_filename)  # Pass the h5_filename to calculate_tau
    log_N_HI = np.log10(N_HI)
    log_tau = np.log10(tau)
    
    # Configuration for Zelda
    my_grids_location = '/Volumes/Hasti-2T-2/main/zelda/Grids/'  # change this part depending to where you download the Grid from zELDA
    Lya.funcs.Data_location = my_grids_location

    # Specify the geometry and prepare input arrays
    Geometry = 'Thin_Shell'  # Other options: 'Galactic Wind' or 'Bicone_X_Slab_In' or 'Bicone_X_Slab_Out'
    V_Arr = np.array(vexp)  # Expansion velocity array in km/s
    logNH_Arr = np.array(log_N_HI)  # Logarithmic of column densities array in cm**-2
    tau_Arr = np.array(log_tau)  # Dust optical depth Array

    # Run Zelda and calculate f_esc
    f_esc_Arr = Lya.RT_f_esc(Geometry, V_Arr, logNH_Arr, tau_Arr)

    # Save the result in the HDF5 file with alpha and beta information
    with h5py.File(h5_filename, 'a') as file:  # Open the file in 'a' (append) mode
        # Get the existing group with the current alpha and beta values
        group_name = f'alpha_{alpha}_beta_{beta}'
        group = file[group_name]

        # Save the f_esc dataset within the group
        dset_f_esc = group.create_dataset('f_esc_Arr', data=f_esc_Arr, compression="gzip", compression_opts=9, chunks=True, maxshape=(None,))

    #print('f_esc is done!')
    return f_esc_Arr


In [8]:
%%time

# Define the range of values for alpha and beta
# Here I just run it for one example of alpha and beta:
alpha_values = np.array([105])
betaa_values = np.array([2e-4])

# Define the range of values for alpha and beta
#alpha_values = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100,
#                        105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180,
#                        185, 190, 195, 200])

#betaa_values = np.array([0.6e-4, 0.7e-4, 0.8e-4, 0.9e-4, 1e-4,
#                        1.1e-4, 1.2e-4, 1.3e-4, 1.4e-4, 1.5e-4, 1.6e-4, 1.7e-4, 1.8e-4, 1.9e-4, 2e-4,
#                        2.1e-4, 2.2e-4, 2.3e-4, 2.4e-4, 2.5e-4, 2.6e-4, 2.7e-4, 2.8e-4, 2.9e-4, 3e-4,
#                        3.1e-4, 3.2e-4, 3.3e-4, 3.4e-4, 3.5e-4, 3.6e-4, 3.7e-4, 3.8e-4, 3.9e-4, 4e-4,
#                        4.1e-4, 4.2e-4, 4.3e-4, 4.4e-4, 4.5e-4, 4.6e-4, 4.7e-4, 4.8e-4, 4.9e-4, 5e-4])

M_Sun = 1.9891e30
m_H = 1.67e-27  # in kg
beta_values = betaa_values / (4 * np.pi * m_H)

# Use itertools.product to generate all combinations
combinations = list(itertools.product(alpha_values, beta_values))

# Loop over all combinations
for alpha, beta in combinations:
    print(f"Calculating for alpha = {alpha}, beta = {beta}")

    # Create an HDF5 file for the current combination with compression and chunking
    h5_filename = f'f_esc_ISM_alpha_{alpha}_beta_{beta}_a=0.258000_z=2.88.h5'

    # Call the function with the current combination
    f_esc_Arr = calculate_f_esc(alpha, beta, dat, h5_filename)

    # Save the result in the HDF5 file
    with h5py.File(h5_filename, 'w') as file:
        dset = file.create_dataset('f_esc_Arr', data=f_esc_Arr, compression="gzip", compression_opts=9, chunks=True, maxshape=(None,))

print("All calculations are done.")


Calculating for alpha = 105, beta = 9.530236113287146e+21


/opt/homebrew/lib/python3.11/site-packages/Lya_zelda/funcs.py:284: RuntimeWarning: invalid value encountered in power
  f_esc = 1./np.cosh( np.sqrt( CCC * (ta**KKK) ) )


All calculations are done.
CPU times: user 1min 6s, sys: 826 ms, total: 1min 7s
Wall time: 1min 7s
